In [1]:
import autogenerate
import importlib
importlib.reload(autogenerate)
autogenerate = autogenerate.autogenerate
import SimplexComplex as SC

In [2]:
active_tables = ["autogenerated_tables","SimplexDart","valid_index_from_tuple","simplex_index_from_valid_index"]

In [3]:
def list_pair_to_array(data, name, comment = None, type_name="int64_t"):
    r = {"name": name, "data": data, "type": type_name}
    if comment is not None: r["comment"] = comment
    return r

In [4]:
def get_switch_data(sc):
    valid_indices = sc.valid_indices()
    all_to_valid = [-1 for _ in sc.all_tuples()]
    for i,v in enumerate(valid_indices):
        all_to_valid[v] = i

    tables = [
                list_pair_to_array(sc.valid_tuples(),"auto_valid_tuples", "Valid tuple local indices", type_name = "int8_t"),
        list_pair_to_array(valid_indices,"auto_valid_tuple_indices", "For each valid tuple encodes the raw tuple index", type_name = "int8_t"),
        list_pair_to_array(all_to_valid,"auto_all_to_valid_tuple_indices", "Index of each tuple according to valid tuple indexing", type_name = "int8_t"),

list_pair_to_array(SC.valid_switch_table(sc),"auto_valid_switch_table", "Valid tuple indices", type_name = "int8_t"),
list_pair_to_array(SC.valid_switch_product_table(sc),"auto_valid_switch_product_table", "Tuple group product using valid tuple indices", type_name = "int8_t"),
list_pair_to_array(SC.valid_switch_inverse_table(sc),"auto_valid_switch_inverse_table", "Tuple group product inverse using valid tuple indices", type_name = "int8_t"),
list_pair_to_array(SC.switches_plus_identity_and_opp(sc),"auto_valid_tuple_switch_indices", "Which tuples are associated with switching. Last two entries are the identity action and opp action", type_name = "int8_t"),
    ]
    dim = len(sc)
    for d in range(1,dim):
        table,max_preservation = SC.face_changing_subdart_tables(sc,d)
        tables.append({"name": f"subdart_preserving_action_{d}", "data": table, "type": "int8_t"})
        tables.append({"name": f"max_subdart_preservation_dimension_{d}", "data": max_preservation, "type": "int8_t"})
    return tables

In [5]:
import switch_tables_2d as st2d
sc2 = st2d.simplex_complex

In [6]:
tri_data = {"namespace": "tri_mesh", "type": "Triangle", "tables": [
list_pair_to_array(st2d.edges,"auto_2d_edges", "lvids for a particular leid"),
list_pair_to_array(st2d.table(0,st2d.simplices),"auto_2d_table_vertex", "vertex switch's (tuple_index -> [lvid,leid])"),
list_pair_to_array(st2d.table(1,st2d.simplices),"auto_2d_table_edge", "edge switch's (tuple_index -> [lvid,leid])"),
list_pair_to_array(st2d.table_ccw(st2d.simplices),"auto_2d_table_ccw", "(tuple_index) -> is_ccw"),
list_pair_to_array(st2d.table_complete_tuple(0,st2d.simplices),"auto_2d_table_complete_vertex", "lvid -> a ccw [lvid,leid]"),
list_pair_to_array(st2d.table_complete_tuple(1,st2d.simplices),"auto_2d_table_complete_edge", "leid -> a ccw [lvid,leid]"),
] + get_switch_data(sc2)}



In [7]:
autogenerate(active_tables, tri_data, use_clang_format=True)


In [8]:
import switch_tables_3d as st3d
sc3 = st3d.simplex_complex

In [9]:
tet_data = {"namespace": "tet_mesh", "type": "Tetrahedron", "tables": [
list_pair_to_array(st3d.edges,"auto_3d_edges", "lvids for a specfic leid"),
list_pair_to_array(st3d.faces,"auto_3d_faces", "lvids for a specific lfid"),
list_pair_to_array(st3d.table(0,st3d.simplices),"auto_3d_table_vertex", "vertex switch's (tuple_index -> [lvid,leid,lfid])"),
list_pair_to_array(st3d.table(1,st3d.simplices),"auto_3d_table_edge", "edge switch's (tuple_index -> [lvid,leid,lfid])"),
list_pair_to_array(st3d.table(2,st3d.simplices),"auto_3d_table_face", "face switch's (tuple_index -> [lvid,leid,lfid])"),
list_pair_to_array(st3d.table_ccw(st3d.simplices),"auto_3d_table_ccw", "(tuple_index) -> is_ccw"),
list_pair_to_array(st3d.table_complete_tuple(0,st3d.simplices),"auto_3d_table_complete_vertex", "lvid -> a ccw [lvid,leid,lfid]"),
list_pair_to_array(st3d.table_complete_tuple(1,st3d.simplices),"auto_3d_table_complete_edge", "evid -> a ccw [lvid,leid,lfid]"),
list_pair_to_array(st3d.table_complete_tuple(2,st3d.simplices),"auto_3d_table_complete_face", "fvid -> a ccw [lvid,leid,lfid]")]
            + get_switch_data(sc3)
           }

In [10]:
autogenerate(active_tables, tet_data, use_clang_format=True)


In [11]:
import switch_tables_1d as st1d
sc1 = st1d.simplex_complex
edge_data = {"namespace": "edge_mesh", "type": "Edge", "tables": 
get_switch_data(sc1)
           }

In [12]:
autogenerate(active_tables,edge_data, use_clang_format=True)


In [13]:
scs = tuple((sc1,sc2,sc3))
data = [[None for _ in scs] for _ in scs]
autogen_tables = []
for r,a in enumerate(scs):
    for c,b in enumerate(scs):
        print(r,c,len(a),len(b))
        if r < c:
            continue
        f,b = SC.switch_normal_subgroup_table(a,b)
        autogen_tables.append({"name": f"subgroup_indices_{r}_{c}", "data": f, "type": "int8_t"})
        if r != c:
            autogen_tables.append({"name": f"subgroup_indices_{c}_{r}", "data": b, "type": "int8_t"})

0 0 1 1
0 1 1 2
0 2 1 3
1 0 2 1
1 1 2 2
1 2 2 3
2 0 3 1
2 1 3 2
2 2 3 3


In [14]:
"""ds = dict()
for dimm1,sc in enumerate(scs):
    dim = dimm1 + 1
    print(":",len(sc),dim)
    for d in range(1,dim):
        print(d,dim)
        ds[(dim,d)] = SC.face_changing_subdart_tables(sc,d)
"""

'ds = dict()\nfor dimm1,sc in enumerate(scs):\n    dim = dimm1 + 1\n    print(":",len(sc),dim)\n    for d in range(1,dim):\n        print(d,dim)\n        ds[(dim,d)] = SC.face_changing_subdart_tables(sc,d)\n'

In [15]:
"""
for pr,pr2 in ds.items():
    dim, bdim = pr
    table,max_preservation = pr2
    autogen_tables.append({"name": f"subdart_preserving_action_{dim}_{bdim}", "data": table, "type": "int8_t"})
    autogen_tables.append({"name": f"max_subdart_preservation_dimension_{dim}_{bdim}", "data": table, "type": "int8_t"})
"""

'\nfor pr,pr2 in ds.items():\n    dim, bdim = pr\n    table,max_preservation = pr2\n    autogen_tables.append({"name": f"subdart_preserving_action_{dim}_{bdim}", "data": table, "type": "int8_t"})\n    autogen_tables.append({"name": f"max_subdart_preservation_dimension_{dim}_{bdim}", "data": table, "type": "int8_t"})\n'

In [16]:
print(autogen_tables)


max_dim = max(map(len,scs))

subgroup_data = {"namespace": "subgroup", "tables": autogen_tables, "dimensions": max_dim}
#print(subgroup_data)

[{'name': 'subgroup_indices_0_0', 'data': (0, 1), 'type': 'int8_t'}, {'name': 'subgroup_indices_1_0', 'data': (-1, 0, -1, 1, -1, -1), 'type': 'int8_t'}, {'name': 'subgroup_indices_0_1', 'data': (1, 3), 'type': 'int8_t'}, {'name': 'subgroup_indices_1_1', 'data': (0, 1, 2, 3, 4, 5), 'type': 'int8_t'}, {'name': 'subgroup_indices_2_0', 'data': (-1, 0, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1), 'type': 'int8_t'}, {'name': 'subgroup_indices_0_2', 'data': (1, 7), 'type': 'int8_t'}, {'name': 'subgroup_indices_2_1', 'data': (-1, 1, -1, 0, -1, -1, -1, 3, -1, 2, -1, -1, -1, 5, -1, 4, -1, -1, -1, -1, -1, -1, -1, -1), 'type': 'int8_t'}, {'name': 'subgroup_indices_1_2', 'data': (3, 1, 9, 7, 15, 13), 'type': 'int8_t'}, {'name': 'subgroup_indices_2_2', 'data': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23), 'type': 'int8_t'}]


In [17]:
autogenerate(["subgroup_transformations"], subgroup_data, use_clang_format=True)
